
HuggingFace Transformers를 활용한 토큰 분류 모델 학습

본 노트북에서는 `klue/roberta-base` 모델을 **KLUE** 내 **NLI** 데이터셋을 활용하여 모델을 훈련하는 예제를 다루게 됩니다.


학습 과정 이후에는 간단한 예제 코드를 통해 모델이 어떻게 활용되는지도 함께 알아보도록 할 것입니다.

모든 소스 코드는 [`huggingface-tutorial`](https://huggingface.co/course/chapter7/2)를 참고하였습니다. 

먼저, 노트북을 실행하는데 필요한 라이브러리를 설치합니다. 모델 훈련을 위해서는 `transformers`가, 학습 데이터셋 로드를 위해서는 `datasets` 라이브러리의 설치가 필요합니다. 그 외 모델 성능 검증을 위해 `scipy`, `scikit-learn`을 추가로 설치해주도록 합니다.

In [ ]:
!pip install  evaluate 
#!pip install accelerate
# To run the training on TPU, you will need to uncomment the following line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
#!apt install git-lfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 67.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 24.0 MB/s eta 0:00:00


In [ ]:
!pip install -U transformers datasets scipy scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 102.8 MB/s eta 0:00:00


In [ ]:
#from huggingface_hub import notebook_login

#notebook_login()

## 문장 분류 모델 학습

노트북을 실행하는데 필요한 라이브러리들을 모두 임포트합니다.

In [ ]:
import random
import logging
from IPython.display import display, HTML

import numpy as np
import pandas as pd
import datasets
from datasets import load_dataset, load_metric, ClassLabel, Sequence,Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from typing import Any, List, Optional, Tuple


학습에 필요한 정보를 변수로 기록합니다.

본 노트북에서는 `klue-roberta-base` 모델을 활용하지만, https://huggingface.co/klue 페이지에서 더 다양한 사전학습 언어 모델을 확인하실 수 있습니다.

학습 태스크로는 `nli`를, 배치 사이즈로는 32를 지정하겠습니다.

In [ ]:
model_checkpoint = "klue/roberta-base"
task = "re"

In [ ]:
batch_size = 16


이제 HuggingFace `datasets` 라이브러리에 등록된 KLUE 데이터셋 중, NLI 데이터를 내려받습니다.

In [ ]:
#['ynat', 'sts', 'nli', 'ner', 're', 'dp', 'mrc', 'wos']
raw_datasets = load_dataset("klue", task)

Generating train split:   0%|          | 0/32470 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/7765 [00:00<?, ? examples/s]

Dataset klue downloaded and prepared to /root/.cache/huggingface/datasets/klue/re/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

다운로드 혹은 로드 후 얻어진 `datasets` 객체를 살펴보면, 훈련 데이터와 검증 데이터가 포함되어 있는 것을 확인할 수 있습니다.

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['guid', 'sentence', 'subject_entity', 'object_entity', 'label', 'source'],
        num_rows: 32470
    })
    validation: Dataset({
        features: ['guid', 'sentence', 'subject_entity', 'object_entity', 'label', 'source'],
        num_rows: 7765
    })
})

In [ ]:
raw_datasets['train'][0]

{'guid': 'klue-re-v1_train_00000',
 'sentence': '〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey Road》에 담은 노래다.',
 'subject_entity': {'word': '비틀즈',
  'start_idx': 24,
  'end_idx': 26,
  'type': 'ORG'},
 'object_entity': {'word': '조지 해리슨',
  'start_idx': 13,
  'end_idx': 18,
  'type': 'PER'},
 'label': 0,
 'source': 'wikipedia'}

각 예시 데이터는 아래와 같이 두 개의 문장과 두 문장의 추론 관계를 라벨로 지니고 있습니다.

In [ ]:
train_id = []
for i,e in enumerate(raw_datasets['train']):
  if e['label'] != 0:
    train_id.append(i)

valid_id = []
for i,e in enumerate(raw_datasets['validation']):
  if e['label'] != 0:
    valid_id.append(i)

In [ ]:
relation_feature = raw_datasets["train"].features["label"]
relation_feature

ClassLabel(names=['no_relation', 'org:dissolved', 'org:founded', 'org:place_of_headquarters', 'org:alternate_names', 'org:member_of', 'org:members', 'org:political/religious_affiliation', 'org:product', 'org:founded_by', 'org:top_members/employees', 'org:number_of_employees/members', 'per:date_of_birth', 'per:date_of_death', 'per:place_of_birth', 'per:place_of_death', 'per:place_of_residence', 'per:origin', 'per:employee_of', 'per:schools_attended', 'per:alternate_names', 'per:parents', 'per:children', 'per:siblings', 'per:spouse', 'per:other_family', 'per:colleagues', 'per:product', 'per:religion', 'per:title'], id=None)

In [ ]:
label_names = relation_feature.names
label_names

['no_relation',
 'org:dissolved',
 'org:founded',
 'org:place_of_headquarters',
 'org:alternate_names',
 'org:member_of',
 'org:members',
 'org:political/religious_affiliation',
 'org:product',
 'org:founded_by',
 'org:top_members/employees',
 'org:number_of_employees/members',
 'per:date_of_birth',
 'per:date_of_death',
 'per:place_of_birth',
 'per:place_of_death',
 'per:place_of_residence',
 'per:origin',
 'per:employee_of',
 'per:schools_attended',
 'per:alternate_names',
 'per:parents',
 'per:children',
 'per:siblings',
 'per:spouse',
 'per:other_family',
 'per:colleagues',
 'per:product',
 'per:religion',
 'per:title']

In [ ]:
label_names.remove("no_relation")

데이터셋을 전반적으로 살펴보기 위한 시각화 함수를 다음과 같이 정의합니다.

In [ ]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."

    picks = []
    
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)

        # 이미 등록된 예제가 뽑힌 경우, 다시 추출
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)

        picks.append(pick)

    # 임의로 추출된 인덱스들로 구성된 데이터 프레임 선언
    df = pd.DataFrame(dataset[picks])

    for column, typ in dataset.features.items():
        # 라벨 클래스를 스트링으로 변환
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i]+f"({str(i)})")

    display(HTML(df.to_html()))

앞서 정의한 함수를 활용해 훈련 데이터를 살펴보도록 합시다.

이처럼 데이터를 살펴보는 것의 장점으로는 각 라벨에 어떠한 문장들이 해당하는지에 대한 감을 익힐 수 있다는데에 있습니다.


In [ ]:
show_random_elements(datasets["train"])

,guid,sentence,subject_entity,object_entity,label,source
0,klue-re-v1_train_16481,"정은경 질병관리본부 중앙방역대책본부(방대본) 본부장은 이날 정례브리핑에서 ""이 교주에 대한 검사는 교회 측으로부터 정보를 확인했다""며 ""음성이라는 것까지 정보를 받았다""고 밝혔다.","{'word': '질병관리본부', 'start_idx': 4, 'end_idx': 9, 'type': 'ORG'}","{'word': '정은경', 'start_idx': 0, 'end_idx': 2, 'type': 'PER'}",org:top_members/employees(10),wikitree
1,klue-re-v1_train_31029,보도에 따르면 아나운서 박찬민 씨가 딸 박민하 양이 곧 있을 초등학교 사격대회에 참가하게 됐다고 밝혔다.,"{'word': '박찬민', 'start_idx': 13, 'end_idx': 15, 'type': 'PER'}","{'word': '아나운서', 'start_idx': 8, 'end_idx': 11, 'type': 'POH'}",per:title(29),wikitree
2,klue-re-v1_train_25703,"그는 등번호 14번을 부여 받았고, 마드리드에서 열린 1-2로 패배한 스페인과의 친선전에서 FC 바르셀로나의 리오넬 메시와 교체 출전하며 마지막 몇분을 뛰며 데뷔전을 치렀다.","{'word': '스페인', 'start_idx': 39, 'end_idx': 41, 'type': 'ORG'}","{'word': '마드리드', 'start_idx': 20, 'end_idx': 23, 'type': 'LOC'}",org:members(6),wikipedia
3,klue-re-v1_train_25143,이 집은 독립운동가로서 해방 후 초대 국회의장을 역임한 해공 신익희 선생(1894～1956)이 국회의장직에서 물러난 1954년 8월부터 1956년 5월 5일 민주당 대통령 후보 자격으로 호남 지역 유세를 위해 전주로 내려가던 중 갑자기 세상을 떠나기까지 약 1년 9개월 여 거주한 곳이다.,"{'word': '신익희', 'start_idx': 34, 'end_idx': 36, 'type': 'PER'}","{'word': '독립운동가', 'start_idx': 5, 'end_idx': 9, 'type': 'POH'}",per:title(29),wikipedia
4,klue-re-v1_train_15197,아이유가 주연으로 출연하는 tvN 드라마 '호텔 델루나'는 엘리트 호텔리어 구찬성(여진구 분) 이 호텔 델루나 사장 장만월(이지은 분)과 함께 호텔을 운영하며 생기는 이야기를 그린다.,"{'word': '호텔 델루나', 'start_idx': 55, 'end_idx': 60, 'type': 'ORG'}","{'word': '여진구', 'start_idx': 46, 'end_idx': 48, 'type': 'PER'}",no_relation(0),wikitree
5,klue-re-v1_train_10238,방송인 전현무와 공개 열애 중인 이혜성 아나운서가 속마음을 밝혔다.,"{'word': '전현무', 'start_idx': 4, 'end_idx': 6, 'type': 'PER'}","{'word': '방송인', 'start_idx': 0, 'end_idx': 2, 'type': 'POH'}",per:title(29),wikitree
6,klue-re-v1_train_06689,"이성미(李聖美, 1959년 12월 25일 ~)는 대한민국의 희극 배우이다.","{'word': '이성미', 'start_idx': 0, 'end_idx': 2, 'type': 'PER'}","{'word': '1959년 12월 25일', 'start_idx': 9, 'end_idx': 21, 'type': 'DAT'}",per:date_of_birth(12),wikipedia
7,klue-re-v1_train_17312,원래는 영화 감독을 지망하였으나 다쓰노코 프로덕션의 《독수리 오형제》를 보고 애니메이션 제작사인 다쓰노코 프로덕션에 입사하였다.,"{'word': '다쓰노코 프로덕션', 'start_idx': 18, 'end_idx': 26, 'type': 'ORG'}","{'word': '애니메이션 제작', 'start_idx': 43, 'end_idx': 50, 'type': 'POH'}",no_relation(0),wikipedia
8,klue-re-v1_train_22825,"1967년 2월 7일, 민중당과 신한당은 재합당하고 신민당으로 창당, 윤보선을 대통령 후보, 유진오를 당 대표로 추대하였다.","{'word': '민중당', 'start_idx': 13, 'end_idx': 15, 'type': 'ORG'}","{'word': '신민당', 'start_idx': 29, 'end_idx': 31, 'type': 'ORG'}",org:alternate_names(4),wikipedia
9,klue-re-v1_train_07919,"에밀리아 클라크(Emilia Clarke, 1986년 10월 23일 ~)는 잉글랜드의 배우이다.","{'word': '에밀리아 클라크', 'start_idx': 0, 'end_idx': 7, 'type': 'PER'}","{'word': '1986년 10월 23일', 'start_idx': 24, 'end_idx': 36, 'type': 'DAT'}",per:date_of_birth(12),wikipedia


훈련 과정 중 모델의 성능을 파악하기 위한 메트릭을 설정합니다.

`datasets` 라이브러리에는 이미 구현된 메트릭을 사용할 수 있는 `load_metric` 함수가 있습니다.


In [ ]:

subject_start_marker = "<subj>"
subject_end_marker = "</subj>"
object_start_marker = "<obj>"
object_end_marker = "</obj>"

def mark_entity_spans(
    text: str,
    subject_range: Tuple[int, int],
    object_range: Tuple[int, int],
) -> str:
    """Adds entity markers to the text to identify the subject/object entities.
    Args:
        text: Original sentence
        subject_range: Pair of start and end indices of subject entity
        object_range: Pair of start and end indices of object entity
    Returns:
        A string of text with subject/object entity markers
    """
    if subject_range < object_range:
        segments = [
            text[: subject_range[0]],
            subject_start_marker,
            text[subject_range[0] : subject_range[1] + 1],
            subject_end_marker,
            text[subject_range[1] + 1 : object_range[0]],
            object_start_marker,
            text[object_range[0] : object_range[1] + 1],
            object_end_marker,
            text[object_range[1] + 1 :],
        ]
    elif subject_range > object_range:
        segments = [
            text[: object_range[0]],
            object_start_marker,
            text[object_range[0] : object_range[1] + 1],
            object_end_marker,
            text[object_range[1] + 1 : subject_range[0]],
            subject_start_marker,
            text[subject_range[0] : subject_range[1] + 1],
            subject_end_marker,
            text[subject_range[1] + 1 :],
        ]
    else:
        raise ValueError("Entity boundaries overlap.")

    marked_text = "".join(segments)

    return marked_text


In [ ]:
df_train = pd.DataFrame(raw_datasets["train"][train_id])
df_valid = pd.DataFrame(raw_datasets["validation"][valid_id])

In [ ]:
def add_tag(dframe):
  for i in dframe.index:
    dframe.loc[i,"subject_entity"]
    subject_range = (int(dframe.loc[i,"subject_entity"]['start_idx']), int(dframe.loc[i,"subject_entity"]['end_idx']))
    object_range = (int(dframe.loc[i,"object_entity"]['start_idx']), int(dframe.loc[i,"object_entity"]['end_idx']))
    sent = dframe.loc[i,"sentence"]
    marked_sent = mark_entity_spans(sent, subject_range, object_range)
    dframe.loc[i,"sentence"] = marked_sent
    dframe.loc[i,"label"] -= 1

    

In [ ]:
add_tag(df_valid)

In [ ]:
add_tag(df_train)

In [ ]:
df_valid

,guid,sentence,subject_entity,object_entity,label,source
0,klue-re-v1_dev_00006,<subj>심은주</subj> <obj>하나금융투자</obj> 연구원은 “매일유업의...,"{'word': '심은주', 'start_idx': 0, 'end_idx': 2, ...","{'word': '하나금융투자', 'start_idx': 4, 'end_idx': ...",17,wikitree
1,klue-re-v1_dev_00007,공개된 영상은 <obj>한국</obj> 경제의 심장부에 서 있는 채이헌 <subj>...,"{'word': '허재', 'start_idx': 29, 'end_idx': 30,...","{'word': '한국', 'start_idx': 8, 'end_idx': 9, '...",16,wikitree
2,klue-re-v1_dev_00009,<obj>김진우</obj> <subj>한국투자증권</subj> 연구원은 “8년 만에...,"{'word': '한국투자증권', 'start_idx': 4, 'end_idx': ...","{'word': '김진우', 'start_idx': 0, 'end_idx': 2, ...",9,wikitree
3,klue-re-v1_dev_00010,<subj>포천시</subj> <obj>관계자</obj>는 “현장 중심의 공정하고 ...,"{'word': '포천시', 'start_idx': 0, 'end_idx': 2, ...","{'word': '관계자', 'start_idx': 4, 'end_idx': 6, ...",9,wikitree
4,klue-re-v1_dev_00012,환경오염 배출시설 및 방지시설 현장 공개 방안에 대해서는 <subj>전라남도</su...,"{'word': '전라남도', 'start_idx': 32, 'end_idx': 3...","{'word': '여수시', 'start_idx': 38, 'end_idx': 40...",5,wikitree
...,...,...,...,...,...,...
3129,klue-re-v1_dev_07755,<subj>한신대학교</subj>는 1940년 <obj>한국</obj> 최초의 신학...,"{'word': '한신대학교', 'start_idx': 0, 'end_idx': 4...","{'word': '한국', 'start_idx': 13, 'end_idx': 14,...",2,wikitree
3130,klue-re-v1_dev_07757,<subj>구</subj> <obj>선수</obj>는 경기 중 80% 정도는 결국 ...,"{'word': '구', 'start_idx': 0, 'end_idx': 0, 't...","{'word': '선수', 'start_idx': 2, 'end_idx': 3, '...",28,wikitree
3131,klue-re-v1_dev_07758,래퍼 <subj>뱃사공</subj>(<obj>김진우</obj>·33)이 11일 인스...,"{'word': '뱃사공', 'start_idx': 3, 'end_idx': 5, ...","{'word': '김진우', 'start_idx': 7, 'end_idx': 9, ...",19,wikitree
3132,klue-re-v1_dev_07762,그러므로 동전의 변덕스러운 회전들은 2명의 예민한 거인 - 7 승 0 패의 <sub...,"{'word': '올라주원', 'start_idx': 42, 'end_idx': 4...","{'word': '샘슨', 'start_idx': 57, 'end_idx': 58,...",25,wikipedia


In [ ]:
#removed no relation labels 
datasets_refined = datasets.DatasetDict(
    {
        "train": Dataset.from_pandas(df_train),
        "validation": Dataset.from_pandas(df_valid),
    }
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=False)

In [ ]:

tokenizer.add_special_tokens(
    {
        "additional_special_tokens": [
            subject_start_marker,
            subject_end_marker,
            object_start_marker,
            object_end_marker,
        ]
    }
)


4

In [ ]:
tokenizer.is_fast

False

In [ ]:
#metric.inputs_description
metric = load_metric("f1")

<ipython-input-24-f6db7250f76c>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("f1")


In [ ]:
def preprocess_function(examples):
    
    return tokenizer(
        examples['sentence'],
        truncation=True,
        return_token_type_ids=False,
    )

In [ ]:
tokenizer.convert_tokens_to_ids("</obj>")

32003

In [ ]:
len(tokenizer)

32004

In [ ]:
# Warning !!!
# tokenizer.vocab_size != len(tokenizer)

AttributeError: ignored

In [ ]:
tokenizer

BertTokenizer(name_or_path='klue/roberta-base', vocab_size=32000, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]', 'additional_special_tokens': ['<subj>', '</subj>', '<obj>', '</obj>']})

In [ ]:
def preprocess_function(examples, tokenizer_type='bert-wp') :
    #max_length = hparams.max_seq_length
    #if max_length is None:
    #    max_length = tokenizer.max_len

    label_map = {label: i+1 for i, label in enumerate(label_names)} #starts with 1
    labels = examples['label']

    def fix_tokenization_error(text: str, tokenizer_type: str) -> Any:
        """Fix the tokenization due to the `obj` and `subj` marker inserted
        in the middle of a word.
        Example:
            >>> text = "<obj>조지 해리슨</obj>이 쓰고 <subj>비틀즈</subj>가"
            >>> tokens = ['<obj>', '조지', '해리', '##슨', '</obj>', '이', '쓰', '##고', '<subj>', '비틀즈', '</subj>', '가']
            >>> fix_tokenization_error(text, tokenizer_type="bert-wp")
            ['<obj>', '조지', '해리', '##슨', '</obj>', '##이', '쓰', '##고', '<subj>', '비틀즈', '</subj>', '##가']
        """
        tokens = tokenizer.tokenize(text)
        # subject
        if text[text.find(subject_end_marker) + len(subject_end_marker)] != " ":
            space_idx = tokens.index(subject_end_marker) + 1
            if tokenizer_type == "xlm-sp":
                if tokens[space_idx] == "▁":
                    tokens.pop(space_idx)
                elif tokens[space_idx].startswith("▁"):
                    tokens[space_idx] = tokens[space_idx][1:]
            elif tokenizer_type == "bert-wp":
                if not tokens[space_idx].startswith("##") and "가" <= tokens[space_idx][0] <= "힣":
                    tokens[space_idx] = "##" + tokens[space_idx]

        # object
        if text[text.find(object_end_marker) + len(object_end_marker)] != " ":
            space_idx = tokens.index(object_end_marker) + 1
            if tokenizer_type == "xlm-sp":
                if tokens[space_idx] == "▁":
                    tokens.pop(space_idx)
                elif tokens[space_idx].startswith("▁"):
                    tokens[space_idx] = tokens[space_idx][1:]
            elif tokenizer_type == "bert-wp":
                if not tokens[space_idx].startswith("##") and "가" <= tokens[space_idx][0] <= "힣":
                    tokens[space_idx] = "##" + tokens[space_idx]

        return tokens

    tokenized_examples = [fix_tokenization_error(text, tokenizer_type) for text in examples['sentence']]

    #you need non-fast tokenizer 
    batch_encoding = tokenizer.batch_encode_plus(
        [(tokenizer.convert_tokens_to_ids(list(tokens)), None) for tokens in tokenized_examples],
        #max_length=max_length,
        #padding="max_length",
        truncation=True,
        return_token_type_ids=False
    )
    return batch_encoding 

In [ ]:
datasets_refined["train"][:5]

{'guid': ['klue-re-v1_train_00002',
  'klue-re-v1_train_00003',
  'klue-re-v1_train_00005',
  'klue-re-v1_train_00007',
  'klue-re-v1_train_00008'],
 'sentence': ['K리그2에서 성적 1위를 달리고 있는 <subj>광주FC</subj>는 지난 26일 <obj>한국프로축구연맹</obj>으로부터 관중 유치 성과와 마케팅 성과를 인정받아 ‘풀 스타디움상’과 ‘플러스 스타디움상’을 수상했다.',
  '균일가 생활용품점 (주)<subj>아성다이소</subj>(대표 <obj>박정부</obj>)는 코로나19 바이러스로 어려움을 겪고 있는 대구광역시에 행복박스를 전달했다고 10일 밝혔다.',
  ': 유엔, 유럽 의회, <subj>북대서양 조약 기구</subj> (<obj>NATO</obj>), 국제이주기구, 세계 보건 기구 (WHO), 지중해 연합, 이슬람 협력 기구, 유럽 안보 협력 기구, 국제 통화 기금, 세계무역기구 그리고 프랑코포니.',
  '<subj>박용오</subj>(朴容旿, <obj>1937년 4월 29일</obj>(음력 3월 19일)(음력 3월 19일) ~ 2009년 11월 4일)는 서울에서 태어난 대한민국의 기업인으로 두산그룹 회장, KBO 총재 등을 역임했다.',
  '중공군에게 온전히 대항할 수 없을 정도로 약해진 국민당은 <obj>타이베이</obj>로 수도를 옮기는 것을 결정해, 남아있는 <subj>중화민국</subj>군의 병력이나 국가, 개인의 재산등을 속속 타이완으로 옮기기 시작해, 12월에는 중앙 정부 기구도 모두 이전해 타이베이 시를 중화민국의 새로운 수도로 삼았다.'],
 'subject_entity': [{'end_idx': 24,
   'start_idx': 21,
   'type': 'ORG',
   'word': '광주FC'},
  {'end_idx': 17, 'start_idx': 13, 'type': 'O

In [ ]:
preprocess_function(datasets_refined["train"][:5])

{'input_ids': [[0, 47, 17665, 2302, 27135, 4610, 21, 2090, 2138, 4214, 2088, 1513, 2259, 32000, 4104, 10904, 32001, 2259, 3625, 4210, 2210, 32002, 3629, 17287, 20212, 32003, 3, 8862, 4415, 4422, 2522, 4852, 4422, 2138, 6157, 2227, 114, 1872, 14198, 2290, 115, 604, 114, 6646, 14198, 2290, 115, 1498, 4812, 2371, 2062, 18, 2], [0, 23306, 2116, 3799, 18319, 2532, 12, 1564, 13, 32000, 27930, 24393, 2024, 32001, 12, 3661, 32002, 6580, 2144, 32003, 13, 793, 1726, 11235, 22328, 8151, 2200, 5117, 2069, 585, 2088, 1513, 2259, 3900, 16955, 2170, 4202, 13473, 2138, 4535, 2371, 4683, 3633, 2210, 3705, 2062, 18, 2], [0, 30, 6125, 16, 4227, 4570, 16, 32000, 23483, 2112, 2221, 8604, 5255, 32001, 12, 32002, 19552, 11216, 32003, 13, 16, 3854, 2052, 2223, 11181, 16, 3665, 5308, 5255, 12, 21534, 13, 16, 16070, 4637, 16, 7814, 4203, 5255, 16, 4227, 5401, 4203, 5255, 16, 3854, 5071, 6898, 16, 28996, 11181, 3673, 4377, 2258, 2208, 2209, 18, 2], [0, 32000, 12365, 2168, 32001, 12, 393, 3, 3, 16, 32002, 20533, 

In [ ]:
ex = [e["label"] for e in datasets_refined["train"]]
print(ex[:2])

[4, 9]


In [ ]:
preprocess_function(datasets_refined["train"][:5])

{'input_ids': [[0, 47, 17665, 2302, 27135, 4610, 21, 2090, 2138, 4214, 2088, 1513, 2259, 32000, 4104, 10904, 32001, 2259, 3625, 4210, 2210, 32002, 3629, 17287, 20212, 32003, 3, 8862, 4415, 4422, 2522, 4852, 4422, 2138, 6157, 2227, 114, 1872, 14198, 2290, 115, 604, 114, 6646, 14198, 2290, 115, 1498, 4812, 2371, 2062, 18, 2], [0, 23306, 2116, 3799, 18319, 2532, 12, 1564, 13, 32000, 27930, 24393, 2024, 32001, 12, 3661, 32002, 6580, 2144, 32003, 13, 793, 1726, 11235, 22328, 8151, 2200, 5117, 2069, 585, 2088, 1513, 2259, 3900, 16955, 2170, 4202, 13473, 2138, 4535, 2371, 4683, 3633, 2210, 3705, 2062, 18, 2], [0, 30, 6125, 16, 4227, 4570, 16, 32000, 23483, 2112, 2221, 8604, 5255, 32001, 12, 32002, 19552, 11216, 32003, 13, 16, 3854, 2052, 2223, 11181, 16, 3665, 5308, 5255, 12, 21534, 13, 16, 16070, 4637, 16, 7814, 4203, 5255, 16, 4227, 5401, 4203, 5255, 16, 3854, 5071, 6898, 16, 28996, 11181, 3673, 4377, 2258, 2208, 2209, 18, 2], [0, 32000, 12365, 2168, 32001, 12, 393, 3, 3, 16, 32002, 20533, 

In [ ]:
datasets_refined["train"]['sentence']

['K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터 관중 유치 성과와 마케팅 성과를 인정받아 ‘풀 스타디움상’과 ‘플러스 스타디움상’을 수상했다.',
 '균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪고 있는 대구광역시에 행복박스를 전달했다고 10일 밝혔다.',
 ': 유엔, 유럽 의회, 북대서양 조약 기구 (NATO), 국제이주기구, 세계 보건 기구 (WHO), 지중해 연합, 이슬람 협력 기구, 유럽 안보 협력 기구, 국제 통화 기금, 세계무역기구 그리고 프랑코포니.',
 '박용오(朴容旿, 1937년 4월 29일(음력 3월 19일)(음력 3월 19일) ~ 2009년 11월 4일)는 서울에서 태어난 대한민국의 기업인으로 두산그룹 회장, KBO 총재 등을 역임했다.',
 '중공군에게 온전히 대항할 수 없을 정도로 약해진 국민당은 타이베이로 수도를 옮기는 것을 결정해, 남아있는 중화민국군의 병력이나 국가, 개인의 재산등을 속속 타이완으로 옮기기 시작해, 12월에는 중앙 정부 기구도 모두 이전해 타이베이 시를 중화민국의 새로운 수도로 삼았다.',
 '특히 김동연 전 경제부총리를 비롯한 김두관 국회의원, 안규백 국회의원, 김종민 국회의원, 오제세 국회의원, 최운열 국회의원, 김정우 국회의원, 권칠승 국회의원, 맹성규 국회의원등 더불어민주당 국회의원 8명이 영상 축하 메세지를 보내 눈길을 끌었다.',
 '하비에르 파스토레는 아르헨티나 클럽 타예레스의 유소년팀에서 축구를 시작하였다.',
 "이른바 'Z세대'로 불리는 1990년대 중반 이후 태어난 세대에게 대표 아이콘으로 통하는 미국 싱어송라이터 빌리 아일리시(본명 빌리 오코널, 19)가 팝 역사를 새로 썼다.",
 '2009년 9월, 미국 프로 야구 필라델피아 필리스 소속의 야구 선수 박찬호는 《MBC 스페셜-박찬호는 당신을 잊지 않았다》 편에서 “최진실 씨의 아픔과 죽음의 고통을 이해합니다. 최진실 씨 사건에 눈물을 흘렸습니다. 저도 죽으려고 마음

In [ ]:
encoded_datasets = datasets_refined.map(preprocess_function, batched=True)

Map:   0%|          | 0/22936 [00:00<?, ? examples/s]

Map:   0%|          | 0/3134 [00:00<?, ? examples/s]

In [ ]:
encoded_datasets

DatasetDict({
    train: Dataset({
        features: ['guid', 'sentence', 'subject_entity', 'object_entity', 'label', 'source', 'input_ids', 'attention_mask'],
        num_rows: 22936
    })
    validation: Dataset({
        features: ['guid', 'sentence', 'subject_entity', 'object_entity', 'label', 'source', 'input_ids', 'attention_mask'],
        num_rows: 3134
    })
})

In [ ]:
num_labels = 29
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifie

In [ ]:
model.resize_token_embeddings(len(tokenizer)) #for the special tokens

Embedding(32004, 768)

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels, average = 'micro')

In [ ]:
metric_name = "f1"

args = TrainingArguments(
    "klue-re",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_datasets["train"],
    eval_dataset=encoded_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1
1,0.642400,0.891564,0.744735
2,0.336900,0.707590,0.822272
3,0.221200,0.846478,0.821315
4,0.160700,0.822379,0.828653
5,0.100400,0.826874,0.835673


TrainOutput(global_step=7170, training_loss=0.34721882100550866, metrics={'train_runtime': 2413.9997, 'train_samples_per_second': 47.506, 'train_steps_per_second': 2.97, 'total_flos': 6493436173156128.0, 'train_loss': 0.34721882100550866, 'epoch': 5.0})

In [ ]:
!cd /content/klue-re && git init && git remote add origin && git pull origin main

Initialized empty Git repository in /content/klue-re/.git/
usage: git remote add [<options>] <name> <url>

    -f, --fetch           fetch the remote branches
    --tags                import all tags and associated objects when fetching
                          or do not fetch any tag at all (--no-tags)
    -t, --track <branch>  branch(es) to track
    -m, --master <branch>
                          master branch
    --mirror[=(push|fetch)]
                          set up remote as a mirror to push to or fetch from



In [ ]:
trainer.evaluate()

{'eval_loss': 0.8268742561340332,
 'eval_f1': 0.8356732610082961,
 'eval_runtime': 16.0758,
 'eval_samples_per_second': 194.951,
 'eval_steps_per_second': 12.192,
 'epoch': 5.0}

In [ ]:
from huggingface_hub import notebook_login

notebook_login()


Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
trainer.push_to_hub()

AttributeError: ignored

In [ ]:
! cd /content/klue-re && rm -rf .git

In [ ]:
! git pull origin main

fatal: not a git repository (or any of the parent directories): .git


In [ ]:
from transformers import pipeline

classifier = pipeline(
    "text-classification",
    model="/content/klue-re/checkpoint-7170",
    return_all_scores=True,
)




In [ ]:
question = "<subj>이순신</subj>은 <obj>1545년</obj>에 태어났다"
classifier(question)


[[{'label': 'LABEL_0', 'score': 1.356785560346907e-05},
  {'label': 'LABEL_1', 'score': 7.049031410133466e-05},
  {'label': 'LABEL_2', 'score': 3.334910070407204e-05},
  {'label': 'LABEL_3', 'score': 3.1606530683347955e-05},
  {'label': 'LABEL_4', 'score': 2.2227599401958287e-05},
  {'label': 'LABEL_5', 'score': 8.83458778844215e-05},
  {'label': 'LABEL_6', 'score': 1.073548355634557e-05},
  {'label': 'LABEL_7', 'score': 2.5850233214441687e-05},
  {'label': 'LABEL_8', 'score': 1.8120286767953075e-05},
  {'label': 'LABEL_9', 'score': 1.988121039175894e-05},
  {'label': 'LABEL_10', 'score': 2.555562423367519e-05},
  {'label': 'LABEL_11', 'score': 0.9990586638450623},
  {'label': 'LABEL_12', 'score': 7.597610238008201e-05},
  {'label': 'LABEL_13', 'score': 6.263943942030892e-05},
  {'label': 'LABEL_14', 'score': 3.8681464502587914e-05},
  {'label': 'LABEL_15', 'score': 1.0385360837972257e-05},
  {'label': 'LABEL_16', 'score': 1.4208227185008582e-05},
  {'label': 'LABEL_17', 'score': 2.553

In [ ]:
subject_start_marker = "<subj>"
subject_end_marker = "</subj>"
object_start_marker = "<obj>"
object_end_marker = "</obj>"


In [ ]:
label_names

['org:dissolved',
 'org:founded',
 'org:place_of_headquarters',
 'org:alternate_names',
 'org:member_of',
 'org:members',
 'org:political/religious_affiliation',
 'org:product',
 'org:founded_by',
 'org:top_members/employees',
 'org:number_of_employees/members',
 'per:date_of_birth',
 'per:date_of_death',
 'per:place_of_birth',
 'per:place_of_death',
 'per:place_of_residence',
 'per:origin',
 'per:employee_of',
 'per:schools_attended',
 'per:alternate_names',
 'per:parents',
 'per:children',
 'per:siblings',
 'per:spouse',
 'per:other_family',
 'per:colleagues',
 'per:product',
 'per:religion',
 'per:title']

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
